# SQL Subqueries - Lab Assignment #2

## Introduction

Now that you've seen how subqueries work, it's time to get some practice writing them! Not all of the queries will require subqueries, but all will be a bit more complex and require some thought and review about aggregates, grouping, ordering, filtering, joins and subqueries. Good luck!  

## Objectives

You will be able to:

* Write subqueries to decompose complex queries

## CRM Database ERD

Once again, here's the schema for the CRM database you'll continue to practice with.

<img src="img/Database-Schema.png" width="600">

## Connect to the Database

As usual, start by importing the necessary packages and connecting to the database `data2.sqlite` in the data folder.

In [39]:
# Your code here; import the necessary packages
import sqlite3
import pandas as pd

In [40]:
# Your code here; create the connection
conn = sqlite3.Connection('data/data2.sqlite')

In [41]:
see_all = """
select * 
from orders
"""

see_all_r = pd.read_sql(see_all, conn)
see_all_r

,orderNumber,orderDate,requiredDate,shippedDate,status,comments,customerNumber
0,10100,2003-01-06,2003-01-13,2003-01-10,Shipped,,363
1,10101,2003-01-09,2003-01-18,2003-01-11,Shipped,Check on availability.,128
2,10102,2003-01-10,2003-01-18,2003-01-14,Shipped,,181
3,10103,2003-01-29,2003-02-07,2003-02-02,Shipped,,121
4,10104,2003-01-31,2003-02-09,2003-02-01,Shipped,,141
...,...,...,...,...,...,...,...
321,10421,2005-05-29,2005-06-06,,In Process,Custom shipping instructions were sent to ware...,124
322,10422,2005-05-30,2005-06-11,,In Process,,157
323,10423,2005-05-30,2005-06-05,,In Process,,314
324,10424,2005-05-31,2005-06-08,,In Process,,141


## Write an Equivalent Query using a Subquery

The following query works using a `JOIN`. Rewrite it so that it uses a subquery instead.

```
SELECT
    customerNumber,
    contactLastName,
    contactFirstName
FROM customers
JOIN orders 
    USING(customerNumber)
WHERE orderDate = '2003-01-31'
;
```

In [42]:
test_qus = """
SELECT
    customerNumber,
    contactLastName,
    contactFirstName
FROM customers
JOIN orders 
    USING(customerNumber)
WHERE orderDate = '2003-01-31'
;
"""

test_qus_res = pd.read_sql(test_qus, conn)
test_qus_res

,customerNumber,contactLastName,contactFirstName
0,141,Freyre,Diego


In [43]:
# Your code here
# Wrote a subquery that takes the column 'customerNumber' from the orders table while also writing a where argument that checks if the order date is exactly Janurary 31, 2003.
qus = """
select customerNumber, contactLastName, contactFirstName
from customers
where customerNumber in(
select customerNumber
from orders 
where orderDate = '2003-01-31'
)
;
"""

qus_res = pd.read_sql(qus, conn)
qus_res


,customerNumber,contactLastName,contactFirstName
0,141,Freyre,Diego


## Select the Total Number of Orders for Each Product Name

Sort the results by the total number of items sold for that product.

In [44]:
# Your code here
# Code is written using sum and count, where it stated that some columns belong to the order details table. We then join the products and order details table through the shared column 'productCode', grouped by product name and ordered by the renamed variable
# total_items_sold in descending order.
q1 = """  
select productName, sum(orderdetails.quantityOrdered) as total_items_sold, count(orderdetails.orderNumber) as order_total
from products 
join orderdetails using(productCode)
group by products.productName
order by total_items_sold desc
;
"""
q1r = pd.read_sql(q1, conn)
q1r


,productName,total_items_sold,order_total
0,1992 Ferrari 360 Spider red,1808,53
1,1937 Lincoln Berline,1111,28
2,American Airlines: MD-11S,1085,28
3,1941 Chevrolet Special Deluxe Cabriolet,1076,28
4,1930 Buick Marquette Phaeton,1074,28
...,...,...,...
104,1999 Indy 500 Monte Carlo SS,855,25
105,1911 Ford Town Car,832,25
106,1936 Mercedes Benz 500k Roadster,824,25
107,1970 Chevy Chevelle SS 454,803,25


## Select the Product Name and the  Total Number of People Who Have Ordered Each Product

Sort the results in descending order.

### A quick note on the SQL  `SELECT DISTINCT` statement:

The `SELECT DISTINCT` statement is used to return only distinct values in the specified column. In other words, it removes the duplicate values in the column from the result set.

Inside a table, a column often contains many duplicate values; and sometimes you only want to list the unique values. If you apply the `DISTINCT` clause to a column that has `NULL`, the `DISTINCT` clause will keep only one NULL and eliminates the other. In other words, the DISTINCT clause treats all `NULL` “values” as the same value.

In [45]:
# Your code here
# Hint: because one of the tables we'll be joining has duplicate customer numbers, you should use DISTINCT
# Selected favored variables, while creating a counted variable of customer numbers. Since a product code is not present within the orders table, we join the order details with both the orders table and products, finally
# grouping them by product names, as well as ordering them in descending order by the new counted variable. 
q2 = """
select productName, count(customerNumber) as customer_total
from orderdetails
join orders using(orderNumber)
join products using(productCode)
group by productName
order by customer_total desc
;
"""

q2r = pd.read_sql(q2, conn)
q2r


,productName,customer_total
0,1992 Ferrari 360 Spider red,53
1,P-51-D Mustang,28
2,HMS Bounty,28
3,F/A 18 Hornet 1/72,28
4,Diamond T620 Semi-Skirted Tanker,28
...,...,...
104,1932 Alfa Romeo 8C2300 Spider Sport,25
105,1917 Grand Touring Sedan,25
106,1911 Ford Town Car,25
107,1957 Ford Thunderbird,24


## Select the Employee Number, First Name, Last Name, City (of the office), and Office Code of the Employees Who Sold Products That Have Been Ordered by Fewer Than 20 people.

This problem is a bit tougher. To start, think about how you might break the problem up. Be sure that your results only list each employee once.

In [46]:
# Your code here
# using distinct, we selected values asked from the task and joined the employee table with the appropriate tables. Finally grouping the results by the employee specific employee number and the products specfic product code,
# adding a having clause that counts the amount of times a customer number has appeared in the orders table (using distinct to clean the data), and setting the range to under 20.
q3 = """
select distinct employees.employeeNumber, employees.firstName, employees.lastName, offices.city, officeCode
from employees
join offices using(officeCode)
join customers on employees.employeeNumber = customers.salesRepEmployeeNumber
join orders using(customerNumber)
join orderdetails using(orderNumber)
join products using(productCode)
group by employees.employeeNumber, products.productCode
having count(distinct orders.customerNumber) < 20
;
"""

q3r = pd.read_sql(q3, conn)
q3r

,employeeNumber,firstName,lastName,city,officeCode
0,1165,Leslie,Jennings,San Francisco,1
1,1166,Leslie,Thompson,San Francisco,1
2,1188,Julie,Firrelli,Boston,2
3,1216,Steve,Patterson,Boston,2
4,1286,Foon Yue,Tseng,NYC,3
5,1323,George,Vanauf,NYC,3
6,1337,Loui,Bondur,Paris,4
7,1370,Gerard,Hernandez,Paris,4
8,1401,Pamela,Castillo,Paris,4
9,1501,Larry,Bott,London,7


## Select the Employee Number, First Name, Last Name, and Number of Customers for Employees Whose Customers Have an Average Credit Limit Over 15K

In [47]:
# Your code here
# Using the same method as previous tasks, after selecting the distinct columns, joined employees and customers, grouping by employee number, and using having to search for results where the average of a an employees customers, whose credit
# limit is over 15000.
q4 = """
select distinct employees.employeeNumber, employees.firstName, employees.lastName, count(customers.customerNumber) as Num_of_Customers
from employees
join customers on employees.employeeNumber = customers.salesRepEmployeeNumber
group by employees.employeeNumber
having avg(customers.creditLimit) > 15000
;
"""

q4r = pd.read_sql(q4, conn)
q4r


,employeeNumber,firstName,lastName,Num_of_Customers
0,1165,Leslie,Jennings,6
1,1166,Leslie,Thompson,6
2,1188,Julie,Firrelli,6
3,1216,Steve,Patterson,6
4,1286,Foon Yue,Tseng,7
5,1323,George,Vanauf,8
6,1337,Loui,Bondur,6
7,1370,Gerard,Hernandez,7
8,1401,Pamela,Castillo,10
9,1501,Larry,Bott,8


## Summary

In this lesson, you got to practice some more complex SQL queries, some of which required subqueries. There's still plenty more SQL to be had though; hope you've been enjoying some of these puzzles!